# Import Libraries

In [ ]:
!pip install dalex

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn
import dalex as dx

from copy import copy

from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

{
    "numpy": np.__version__,
    "pandas": pd.__version__,
    "matplotlib": matplotlib.__version__,
    "seaborn": sns.__version__,
    "sklearn": sklearn.__version__,
    "dalex": dx.__version__,
}

In [ ]:
df = pd.read_csv("./stackoverflow_full.csv", index_col=0)
target = "Employed"

### Insert here the "no changes model cells"

## ML prerequisites

In [ ]:
#split your data set in 2 parts : training and testing

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=target),
    df[target],
    test_size=0.3,
    random_state=42
)

In [ ]:
# Protected attribute is 0 if a man or non binary and 0 if a woman plus the age

protected = (pd.Series(np.where(X_test["Gender"] == "Woman", '1', '0'), index=X_test.index) 
             + '_' 
             + X_test.Age)
protected_train = (pd.Series(np.where(X_train["Gender"] == "Woman", '1', '0').astype(str), index=X_train.index) 
                   + '_' 
                   + X_train.Age)

# Privileged population is men under 35 years old
privileged = '0_<35'

In [ ]:
preprocessor = make_column_transformer(
      ("passthrough", make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore"), make_column_selector(dtype_include=object))
)

#You can change the Decision tree hyperparameters or the classifier below

clf_decisiontree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=10, random_state=123))
])

In [ ]:
# clf_decisiontree.fit(df.drop(columns=[target]), df[target])
clf_decisiontree.fit(X_train, y_train)

In [ ]:
# exp_decisiontree = dx.Explainer(clf_decisiontree, df.drop(columns=[target]), df[target], verbose=False)
exp_decisiontree = dx.Explainer(clf_decisiontree, X_test, y_test, verbose=True)

In [ ]:
exp_decisiontree.model_performance().result

In [ ]:
fairness_decisiontree = exp_decisiontree.model_fairness(protected=protected, privileged=privileged)

In [ ]:
fairness_decisiontree.fairness_check(epsilon = 0.8) # default epsilon

In [ ]:
fairness_decisiontree.plot(verbose=False)

###

### Strategy 1: Pre-processing: Resampling

The next thing that can come to mind is to resample the data. Dalex provide 2 types of resampling methods and 1 reweighting method. In this tutorial only the basic resampling is showed.

#### Training

In [ ]:
preprocessor = make_column_transformer(
      ("passthrough", make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore"), make_column_selector(dtype_include=object))
)

#You can change the Decision tree hyperparameters or the classifier below

clf_decisiontree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=10, random_state=123))
])

In [ ]:
#split your data set in 2 parts : training and testing

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=target),
    df[target],
    test_size=0.3,
    random_state=42
)

In [ ]:
# clf_decisiontree.fit(df.drop(columns=[target]), df[target])
clf_decisiontree.fit(X_train, y_train)

In [ ]:
from dalex.fairness import resample
clf_resampled = copy(clf_decisiontree) # Create a copy to not alter the main object

# Resampling observations
indices_uniform = resample(protected_train, y_train, verbose = False)

# Re-fit model with resampled data
clf_resampled.fit(X_train.reset_index(drop=True).iloc[indices_uniform, :], y_train.reset_index(drop=True)[indices_uniform])

In [ ]:
exp_decisiontree_resampled = dx.Explainer(clf_resampled, X_test, y_test, verbose=True)

#### Algorithmic performance

In [ ]:
exp_decisiontree_resampled.model_performance().result

#### Fairness performance

In [ ]:
fairness_decisiontree_resampled = exp_decisiontree_resampled.model_fairness(
    protected, privileged, label='DecisionTreeClassifier_resampled')

fairness_decisiontree_resampled.fairness_check(epsilon = 0.8)


__Compare performance of the first model and the resampled one (visually)__

In [ ]:
fairness_decisiontree.plot([fairness_decisiontree_resampled])

In [ ]:
fairness_decisiontree.plot([fairness_decisiontree_resampled], type='radar')

- Is this strategy effective in terms of algorithmic performance?
- What comment can you make based on the fairness metric result?
- Could you think of a way to improve this strategy (not necessarly on python but as a complementary idea to this solution)?

If you have more time or wish to compare, feel free to try the other strategies and compare the results (you can plot the fairness metrics to have a visual comparison).